In [19]:
import gradio as gr
from ultralytics import YOLO
import cv2
import tempfile
import os
import easyocr
from paddleocr import PaddleOCR, TextRecognition
import re


# --- Cargar modelos ---

ocr_model = TextRecognition(model_name="PP-OCRv5_server_rec")
# reader = PaddleOCR(use_textline_orientation=True, lang="en")  # Ajustar idioma según necesites

model = YOLO("/home/gnz/GitHub/yolo11_container/runs/detect/train2/weights/best.pt")

# Inicializar EasyOCR ---
# reader = easyocr.Reader(['en','es'])  # podés agregar ['en', 'es'] si querés español

Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/gnz/.paddlex/official_models/PP-OCRv5_server_rec`.


In [ ]:
def process_image(image):
    if image is None:
        return None, [], ""

    # Convertir BGR a RGB
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = model.predict(img, conf=0.25, verbose=False)

    annotated = results[0].plot()
    crops, texts = [], []

    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        crop = img[y1:y2, x1:x2]

        # --- Preprocesamiento estilo PaddleOCR app.py ---
        gray = cv2.cvtColor(crop, cv2.COLOR_RGB2GRAY)
        _, binarized = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        resized = cv2.resize(binarized, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
        crop_rgb = cv2.cvtColor(resized, cv2.COLOR_GRAY2RGB)

        # --- OCR con TextRecognition ---
        ocr_result = ocr_model.predict(crop_rgb)  # Lista de dicts: {'text': ..., 'score': ...}
        texts_in_crop = [res['text'] for res in ocr_result if res.get('text')]

        crops.append(crop)
        texts.append(" ".join(texts_in_crop) if texts_in_crop else "(sin texto)")

    return annotated, crops, "\n".join(texts)




# --- Interfaz Gradio ---
with gr.Blocks() as demo:
    gr.Markdown("## 🚢 Detección de IDs y OCR en Contenedores (Solo imágenes)")

    with gr.Row():
        inp_img = gr.Image(type="numpy", label="Sube una imagen")
        out_img = gr.Image(type="numpy", label="Detecciones")
    gallery = gr.Gallery(label="Crops detectados", columns=3, height="auto")
    out_texts = gr.Textbox(label="Resultados OCR", lines=6)

    inp_img.change(process_image, inputs=inp_img, outputs=[out_img, gallery, out_texts])

demo.launch()


* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.


: 